Imagine we’re building an agent to automate accounts payable processing. Every day, the agent receives dozens of emails with attached invoices from different vendors, each using their own unique format and layout. Some are PDFs, others are scanned images that have been converted to text, and a few even arrive as plain text in the email body. Our agent needs to understand each invoice, extract key information like the invoice number, date, amount, and line items, and then insert this data into the company’s accounting database. Without automation, this would be a tedious manual task requiring someone to read each invoice and transcribe the important details.

This is where the computational power of large language models becomes transformative. Through self-prompting, our agent can use an LLM as a universal parser that understands the natural structure of invoices, regardless of their format. The LLM can read an invoice like a human would, identifying key information based on context and common patterns, then output that information in a structured format our agent can process. We don’t need to write separate parsers for each vendor’s invoice format or maintain complex rules about where to find specific information - the LLM can handle all of that complexity for us.

Here’s how we can implement this capability as a reusable tool for our agent:

In [ ]:
@register_tool()
def prompt_llm_for_json(action_context: ActionContext, schema: dict, prompt: str):
    """
    Have the LLM generate JSON in response to a prompt. Always use this tool when you need structured data out of the LLM.
    This function takes a JSON schema that specifies the structure of the expected JSON response.

    Args:
        schema: JSON schema defining the expected structure
        prompt: The prompt to send to the LLM

    Returns:
        A dictionary matching the provided schema with extracted information
    """
    generate_response = action_context.get("llm")

    # Try up to 3 times to get valid JSON
    for i in range(3):
        try:
            # Send prompt with schema instruction and get response
            response = generate_response(Prompt(messages=[
                {"role": "system",
                 "content": f"You MUST produce output that adheres to the following JSON schema:\n\n{json.dumps(schema, indent=4)}. Output your JSON in a ```json markdown block."},
                {"role": "user", "content": prompt}
            ]))

            # Check if the response has json inside of a markdown code block
            if "```json" in response:
                # Search from the front and then the back
                start = response.find("```json")
                end = response.rfind("```")
                response = response[start+7:end].strip()

            # Parse and validate the JSON response
            return json.loads(response)

        except Exception as e:
            if i == 2:  # On last try, raise the error
                raise e
            print(f"Error generating response: {e}")
            print("Retrying...")

You may have noticed a new action_context parameter above. Don’t worry about that for now, we will talk about this architectural choice in a later section. For now, just know that it is a context object that contains the LLM and other useful information.

With this tool in place, we can extract structured data from any text input. For example, an agent processing business documents could extract information in standardized formats:

From an invoice:

In [ ]:
invoice_schema = {
    "type": "object",
    "properties": {
        "invoice_number": {"type": "string"},
        "date": {"type": "string"},
        "amount": {"type": "number"},
        "line_items": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "description": {"type": "string"},
                    "quantity": {"type": "number"},
                    "unit_price": {"type": "number"}
                }
            }
        }
    }
}

extracted_data = prompt_llm_for_json(
    action_context=context,
    schema=invoice_schema,
    prompt="Extract invoice details from this text: 'INVOICE #1234...'"
)

The tool works by combining several key elements:

- A system message that firmly instructs the LLM to adhere to the provided JSON schema
- Retry logic to handle potential parsing failures
- Support for both direct JSON output and markdown-formatted JSON
- Error handling to ensure we either get valid JSON or fail explicitly

What makes this approach especially valuable is that it creates a reliable bridge between unstructured text and structured data processing. The agent can use this tool whenever it needs to convert natural language information into a format that other systems can process programmatically. This enables workflows where the agent can:

Receive unstructured text input from various sources
Use prompt_llm_for_json to extract relevant information in a structured format
Pass the structured data to other APIs or services for further processing
Make decisions based on the processed results

# Balancing Flexibility and Reliability in Data Extraction

Below is an example to make it more reliable BUT less flexible

Here we specify it is for invoice and tell it more information what each element of an invoice would look like.


In [ ]:
@register_tool(tags=["document_processing", "invoices"])
def extract_invoice_data(action_context: ActionContext, document_text: str) -> dict:
    """
    Extract standardized invoice data from document text. This tool enforces a consistent
    schema for invoice data extraction across all documents.

    Args:
        document_text: The text content of the invoice to process

    Returns:
        A dictionary containing extracted invoice data in a standardized format
    """
    # Define a fixed schema for invoice data
    invoice_schema = {
        "type": "object",
        "required": ["invoice_number", "date", "amount"],  # These fields must be present
        "properties": {
            "invoice_number": {"type": "string"},
            "date": {"type": "string", "format": "date"},
            "amount": {
                "type": "object",
                "properties": {
                    "value": {"type": "number"},
                    "currency": {"type": "string"}
                },
                "required": ["value", "currency"]
            },
            "vendor": {
                "type": "object",
                "properties": {
                    "name": {"type": "string"},
                    "tax_id": {"type": "string"},
                    "address": {"type": "string"}
                },
                "required": ["name"]
            },
            "line_items": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "description": {"type": "string"},
                        "quantity": {"type": "number"},
                        "unit_price": {"type": "number"},
                        "total": {"type": "number"}
                    },
                    "required": ["description", "total"]
                }
            }
        }
    }

    # Create a focused prompt that guides the LLM in invoice extraction
    extraction_prompt = f"""
    Extract invoice information from the following document text.
    Focus on identifying:
    - Invoice number (usually labeled as 'Invoice #', 'Reference', etc.)
    - Date (any dates labeled as 'Invoice Date', 'Issue Date', etc.)
    - Amount (total amount due, including currency)
    - Vendor information (company name, tax ID if present, address)
    - Line items (individual charges and their details)

    Document text:
    {document_text}
    """

    # Use our general extraction tool with the specialized schema and prompt
    return prompt_llm_for_json(
        action_context=action_context,
        schema=invoice_schema,
        prompt=extraction_prompt
    )

This specialized approach offers several advantages:

- Data Consistency: The fixed schema ensures that invoice data is always structured the same way, making it easier to work with downstream systems like databases or accounting software.

- Required Fields: We can specify which fields are required, ensuring critical information is always extracted or an error is raised if it can’t be found.

- Field Validation: The schema can include format specifications (like ensuring dates are properly formatted) and field-specific constraints.

- Focused Prompting: We can provide detailed guidance to the LLM about where to look for specific information, improving extraction accuracy.

However, this specialization also means we need to create and maintain separate extraction tools for each type of document we want to process. If we later need to handle purchase orders, receipts, or contracts, we’ll need to implement new tools for each.



# The choice between these approaches often depends on your specific needs:

**Use specialized tools when:**
- Data consistency is critical
- You have a well-defined set of document types
- You need to enforce specific validation rules
- The extracted data feeds into other systems with strict requirements

**Use the general-purpose approach when:**

- You need to handle a wide variety of document types
- Document formats and requirements change frequently
- You’re prototyping or exploring new use cases
- The downstream systems are flexible about data format